In [37]:
%reload_ext autoreload
%autoreload 2

In [38]:
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_get_app_dir
from k12libs.utils.nb_easy import k12ai_get_top_dir
from k12libs.utils.nb_widget import k12ai_schema_parse

import os
import json
import _jsonnet

In [39]:
schema_dir = os.path.join(k12ai_get_top_dir(), 'k12libs', 'templates', 'schema')
!ls $schema_dir 

basic.jsonnet	   k12ai_basic_type.jsonnet
default.libsonnet  k12ai_complex_type.jsonnet


In [40]:
# basic schema template file
basic_file = os.path.join(schema_dir, 'k12ai_layout_type.jsonnet')
!cat $basic_file

// @file k12ai_basic_type.jsonnet
// @brief
// @author QRS
// @version 1.0
// @date 2019-12-23 19:04

local default_schema = import 'default.libsonnet';

{
    description: |||
        K12 Data Template Model for Basic Type
    |||,

    type: 'page',
    objs: [
        default_schema.bool('k12.test', '5'),
        {
            type: 'H',
            objs: [
                default_schema.int('k12.test.basic', '1'),
                default_schema.float('k12.test.basic', '1'),
                default_schema.string('k12.test.basic', '1'),
                default_schema.string_enum('k12.test.basic', '1'),
                default_schema.int_array('k12.test.array', '1'),
                default_schema.float_array('k12.test.array', '1'),
                default_schema.string_array('k12.test.array', '1'),
            ],
        },
        {
            type: 'H',
            objs: [
                default_schema.image('_js.image.k12.test.image', 0, '00000.jpg'),
                default_sch

In [41]:
basic_json = _jsonnet.evaluate_file(basic_file)
k12ai_print(basic_json, indent=2)

{
  "description": "K12 Data Template Model for Basic Type\n",
  "objs": [
    {
      "_id_": "k12.test.k12bool",
      "default": true,
      "name": {
        "cn": "布尔-5",
        "en": "Bool5"
      },
      "type": "bool"
    },
    {
      "objs": [
        {
          "_id_": "k12.test.basic.k12int",
          "default": 100,
          "max": 120,
          "min": 0,
          "name": {
            "cn": "整型-1",
            "en": "Int-1"
          },
          "type": "int"
        },
        {
          "_id_": "k12.test.basic.k12float",
          "default": 200,
          "max": 300,
          "min": 0,
          "name": {
            "cn": "浮点-1",
            "en": "Float1"
          },
          "type": "float"
        },
        {
          "_id_": "k12.test.basic.k12string",
          "default": "string",
          "name": {
            "cn": "字符串-1",
            "en": "String-1"
          },
          "type": "string"
        },
        {
          "_id_": "k12.test.basi

In [42]:
k12ai_schema_parse(json.loads(basic_json), lan='cn', debug=False);

Box(children=(VBox(children=(Checkbox(value=True, description='布尔-5'), HBox(children=(BoundedIntText(value=100…